In [1]:
import os
import sys
import nltk
from tqdm import trange
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
from src.preprocessing.corpus_preprocessor import CorpusPreprocessor
from src.preprocessing.functions import *
from src.preprocessing.consts import EMAIL_REGEX
from src.ds_loaders.xsum import XSumLoader

[nltk_data] Downloading package punkt to /home/pasha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/pasha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/pasha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/pasha/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
2022-05-26 14:14:02.206193: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


## Test my ds loader

In [2]:
loader = XSumLoader()
loader.load()

Using custom data configuration default
Reusing dataset xsum (/home/pasha/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
orig_documents, summaries = loader.X_val, loader.y_val

In [4]:
print(orig_documents[219])

# print(loader.X_train[4])

## Test my preprocessing

In [5]:
preprocessor = CorpusPreprocessor(verbose=False).\
    add(to_lower()).\
    add(expand_contractions()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_by_regex([r"\d"])).\
    add(remove_symbols(string.punctuation)).\
    add(lemmatize()).\
    add(remove_words(set(stopwords.words('english'))))

In [6]:
for idx in trange(len(orig_documents)):
    orig_documents[idx] = nltk.sent_tokenize(orig_documents[idx])

100%|██████████| 11332/11332 [00:03<00:00, 2897.87it/s]


In [7]:
print(orig_documents[219])

[]


In [8]:
documents = []
for idx in trange(len(orig_documents)):
    doc = orig_documents[idx]
    if len(doc) == 0:
        summaries = summaries[:idx] + summaries[idx+1:]
        continue
    documents.append(preprocessor.transform(doc))

100%|██████████| 11332/11332 [00:20<00:00, 540.43it/s]


## test some embedings

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
cur_doc = documents[0]
X = vectorizer.fit_transform(cur_doc)
# print(cur_doc[3])
print(X.toarray())

print(vectorizer.get_feature_names_out())

[[0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 0 0 0 1 0 1 0 0
  1 2 0 0 0 1 0]
 [0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0
  1 0 0 1 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0
  0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0
  0 0 0 0 1 0 0]
 [0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1
  0 0 1 1 0 0 1]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
  0 0 0 0 0 0 0]]
['allegedly' 'appearing' 'bail' 'bailey' 'bexley' 'bright' 'brother'
 'charge' 'charged' 'charity' 'defender' 'denied' 'due' 'earlier' 'efe'
 'elder' 'exreading' 'foundation' 'four' 'fraudulent' 'greater' 'jointly'
 'july' 'kent' 'manchester' 'money' 'mr' 'nigerian' 'offence' 'old'
 'place' 'raise' 'relates' 'relating' 'released' 'sam' 'sodje' 'sport'
 'stand' 'stephen' 'took' 'trading' 'trial']


In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf=False)
tfidf = transformer.fit_transform(X)
print(tfidf.toarray())

[[0.         0.         0.         0.         0.         0.
  0.         0.18974499 0.         0.25241555 0.25241555 0.18974499
  0.         0.         0.         0.         0.25241555 0.25241555
  0.         0.25241555 0.         0.         0.         0.
  0.         0.25241555 0.         0.25241555 0.         0.
  0.         0.25241555 0.         0.25241555 0.         0.
  0.18974499 0.50483109 0.         0.         0.         0.25241555
  0.        ]
 [0.         0.         0.         0.         0.         0.27898225
  0.37112683 0.         0.37112683 0.         0.         0.
  0.         0.         0.27898225 0.37112683 0.         0.
  0.         0.         0.         0.37112683 0.         0.
  0.         0.         0.37112683 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.27898225 0.         0.         0.27898225 0.         0.
  0.        ]
 [0.         0.40389105 0.         0.40389105 0.         0.
  0.         0.         0.         0.   

In [11]:
from gensim.models import Word2Vec
# print(documents[0])
# tekenized0 = [sentence.split() for sentence in documents[0]]
#
model = Word2Vec(
    window=10,
    min_count=2,
    workers=4,
)
model.build_vocab(documents, progress_per=1000)
model.train(documents, total_examples=model.corpus_count, epochs=model.epochs)

# model.save("./word2vec2.model")


# model = word2vec.Word2Vec(tekenized0, min_count=1)
# model.pre

(36440, 1056600)

In [12]:
model.wv.most_similar("laugh")

[('also said wa enough affordable housing proposed added residential element wa contrary scottish planning policy resident would travel basic amenity service',
  0.3110412657260895),
 ('foul john akinde barnet', 0.28199031949043274),
 ('conceded james mckeown', 0.2805235981941223),
 ('inquest continues', 0.27203214168548584),
 ('grateful thought prayer guiding next stop', 0.26940277218818665),
 ('last month human right group amnesty international said men boy died nigerian military custody detained suspected militant',
  0.2688613533973694),
 ('police appealing witness', 0.26770326495170593),
 ('foul james perch queen park ranger', 0.2583902180194855),
 ('dob', 0.25739988684654236),
 ('take time', 0.2560594379901886)]

## Test metrics

In [47]:
loader = XSumLoader()
loader.load()
orig_documents, summaries = loader.X_val, loader.y_val

Using custom data configuration default
Reusing dataset xsum (/home/pasha/.cache/huggingface/datasets/xsum/default/1.2.0/32c23220eadddb1149b16ed2e9430a05293768cfffbdfd151058697d4c11f934)


  0%|          | 0/3 [00:00<?, ?it/s]

In [48]:
preprocessor = CorpusPreprocessor(verbose=False).\
    add(to_lower()).\
    add(expand_contractions()).\
    add(replace_by_regex([(EMAIL_REGEX, "EMAIL")])).\
    add(remove_by_regex([r"\d"])).\
    add(remove_symbols(string.punctuation.replace('.', ''))).\
    add(lemmatize()).\
    add(remove_words(set(stopwords.words('english'))))

In [49]:
documents = []
for idx in trange(len(orig_documents)):
    doc = orig_documents[idx]
    if len(doc) == 0:
        summaries = summaries[:idx] + summaries[idx+1:]
        continue
    documents.append(preprocessor.transform(doc))

100%|██████████| 11332/11332 [00:17<00:00, 651.27it/s]


In [50]:
orig_documents[1]

'Voges was forced to retire hurt on 86 after suffering the injury while batting during the County Championship draw with Somerset on 4 June.\nMiddlesex hope to have the Australian back for their T20 Blast game against Hampshire at Lord\'s on 3 August.\nThe 37-year-old has scored 230 runs in four first-class games this season at an average of 57.50.\n"Losing Adam is naturally a blow as he contributes significantly to everything we do," director of cricket Angus Fraser said.\n"His absence, however, does give opportunities to other players who are desperate to play in the first XI.\n"In the past we have coped well without an overseas player and I expect us to do so now."\nDefending county champions Middlesex are sixth in the Division One table, having drawn all four of their matches this season.\nVoges retired from international cricket in February with a Test batting average of 61.87 from 31 innings, second only to Australian great Sir Donald Bradman\'s career average of 99.94 from 52 Te

In [56]:
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
txt = orig_documents[1]
my_parser = PlaintextParser.from_string(txt ,Tokenizer('english'))
summarizer = LexRankSummarizer()
lexrank_summary = summarizer(my_parser.document, sentences_count=2)
my_sum = str(lexrank_summary[0])
orig_sum = summaries[1]
# print(lexrank_summary)
# print(" ".join(list(map(lambda x: str(x), lexrank_summary))))
# print("\n\n\n")
# print(txt)
# print(orig_documents[1])
# print(lexrank_summary)
# print(". ".join(list(lexrank_summary)))
# print(". ".join(my_sum))
# print(orig_sum)

(<Sentence: Chris Poole - known as "moot" online - created the site in 2003.>, <Sentence: Mr Poole shared news of his new position on blogging site Tumblr.>)
Chris Poole - known as "moot" online - created the site in 2003. Mr Poole shared news of his new position on blogging site Tumblr.




Chris Poole - known as "moot" online - created the site in 2003.
It has gone on to be closely associated with offensive and often illegal activity, including instances where the images of child abuse were shared.
It was widely credited as being the first place where leaked images of nude celebrities were posted following 2014's well-publicised security breach affecting Apple's iCloud service. That incident prompted a policy change on the site.
However, 4chan has also been the rallying point for many instances of online activism from the likes of Anonymous, the loosely organized hacktivism group.
Mr Poole shared news of his new position on blogging site Tumblr.
"When meeting with current and former 

In [20]:
from datasets import load_metric
import numpy as np
bleu = load_metric("bleu")
pred = [my_sum.split()]
ref = [[orig_sum.split()]]
print(pred)
print(ref)
print(bleu.compute(predictions=pred, references=ref))
print(np.mean(bleu.compute(predictions=pred, references=ref)['precisions']))

[["['Voges", 'was', 'forced', 'to', 'retire', 'hurt', 'on', '86', 'after', 'suffering', 'the', 'injury', 'while', 'batting', 'during', 'the', 'County', 'Championship', 'draw', 'with', 'Somerset', 'on', '4', 'June.']]
[[['Middlesex', 'batsman', 'Adam', 'Voges', 'will', 'be', 'out', 'until', 'August', 'after', 'suffering', 'a', 'torn', 'calf', 'muscle', 'in', 'his', 'right', 'leg.']]]
{'bleu': 0.0, 'precisions': [0.08333333333333333, 0.043478260869565216, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.263157894736842, 'translation_length': 24, 'reference_length': 19}
0.03170289855072464


In [5]:
from datasets import load_metric

In [8]:
bleu = load_metric("bleu")

pred = ["I have thirty six years".split()]
ref = [[
    "I am thirty six years old".split(),
    # "I am thirty six".split()
]]
print(bleu.compute(predictions=pred, references=ref, max_order=2))

{'bleu': 0.5178107940302672, 'precisions': [0.8, 0.5], 'brevity_penalty': 0.8187307530779819, 'length_ratio': 0.8333333333333334, 'translation_length': 5, 'reference_length': 6}


In [10]:
from pprint import pprint
rouge = load_metric("rouge")
result = rouge.compute(predictions=pred, references=ref)
pprint(result)
pprint("=" * 20)
print(result['rouge2'])
pprint(result['rouge2'][1][2])

{'rouge1': AggregateScore(low=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272), mid=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272), high=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272)),
 'rouge2': AggregateScore(low=Score(precision=0.5, recall=0.4, fmeasure=0.4444444444444445), mid=Score(precision=0.5, recall=0.4, fmeasure=0.4444444444444445), high=Score(precision=0.5, recall=0.4, fmeasure=0.4444444444444445)),
 'rougeL': AggregateScore(low=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272), mid=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272), high=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272)),
 'rougeLsum': AggregateScore(low=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272), mid=Score(precision=0.8, recall=0.6666666666666666, fmeasure=0.7272727272727272), high=Score(precision=0.8, reca

In [11]:
from src.metrics import BLEU, ROUGE

In [13]:
bm = BLEU(max_order=2)
rm = ROUGE(subtype="rouge2", submetric="fmeasure")

pred = ["I have thirty six years"]
ref = ["I am thirty six years old"]
bm.update_state(pred, ref)
rm.update_state(pred, ref)
print(f"{bm}: {bm.result()}")
print(f"{rm}: {float(rm.result())}")

BLEU-2: 0.5178107940302672
<src.metrics.ROUGE.ROUGE object at 0x7fabb98c0520>: 0.4444444444444445


In [1]:
from src.metrics import GloveCosineSimilarity

2022-05-26 17:01:55.517661: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
metric = GloveCosineSimilarity()

DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt
DEBUG: Loaded GloVe embedings from /home/pasha/Documents/ML/ML_project_summarization/src/embedings/../../data/glove/glove.6B.100d.txt
0.9187776651788369


In [7]:
pred = ["I have thirty six years"]
ref = ["I am thirty six years old"]
ref = ["He is "]
metric.reset_state()
metric.update_state(pred, ref)
print(metric.result())

0.6254994309841915
